### Generating the dataset

In [1]:
srand(1)

# number of users
d1 = 10;

# number of items
d2 = 13;

# number of queries
n = 300;

In [2]:
# generate the hidden variables as a 1-rank
ThetaS = rand(d1) * rand(d2)'

# Need to have the sum of rows equaling 0
for i = 1:d1
    ThetaS[i,:] -= mean(ThetaS[i,:])
end

# Need to make the Frobenius norm < 1
ThetaS = ThetaS / vecnorm(ThetaS);

In [3]:
y = []
X = []

for i = 1:n
    Xi = zeros(d1, d2)
    
    lin = rand(1:d1)
    c1 = rand(1:d2)
    c2 = rand(1:d2)
    while c2 == c1
        c2 = rand(1:d2)
    end
    
    Xi[lin,c1] = 1
    Xi[lin,c2] = -1
    Xi = Xi * sqrt(d1 * d2)
    
    push!(X, Xi)
    
    if ThetaS[lin,c1] > ThetaS[lin,c2]
        push!(y, 1)
    else
        push!(y, 0)
    end
end

### Helper functions

In [4]:
function nucNorm(A)
    return sum(svd(A)[2])
end

nucNorm (generic function with 1 method)

In [5]:
# make sure the matrix is centered and with norm < 1

function adjust(X)
    ans = copy(X)
    
    for i = 1:d1
        ans[i,:] -= mean(ans[i,:])
    end
    
    return ans / vecnorm(ans)
end

adjust (generic function with 1 method)

In [6]:
# The loss function

function loss(Theta)
    ans = 0
    
    for i = 1:n
        ans = ans + log(1 + e^(trace(Theta' * X[i]))) - y[i] * trace(Theta' * X[i])
    end
    
    return ans / n
end

loss (generic function with 1 method)

In [7]:
# Compute the derivative of the loss function. This will be the 'v' in the proximal step

function deltaF(Theta)
    
    ans = zeros(d1,d2)
    
    for i = 1:n
        t1 = 1 / (1 + e^trace(Theta' * X[i]))
        t2 = e^trace(Theta' * X[i]) * X[i]
        t3 = y[i] * X[i]
        
        ans = ans + t1 * t2 - t3
    end
    
    return ans / n
    
end

deltaF (generic function with 1 method)

In [8]:
# The proximal objective

function proxObj(lambda, X, V)
    return nucNorm(X) + 1 / (2 * lambda) * vecnorm(X - V)^2
end

proxObj (generic function with 1 method)

In [9]:
# The proximal function
# return the matrix X that minimizes lambda * ||X||_nuc + 1/(2 lambda) || X - V ||_2^2

function prox(lambda, V)
    
    X = adjust(rand(d1,d2))
    Xmin = copy(X)
    
    step = 0.05
    eps = 0.2
    numSteps = 0
    
    while numSteps < 200
        numSteps += 1
    
        # a subgradient of lambda * ||X||_nuc is AB' where X = ASB'
        A,_,B = svd(X)
        t1 = lambda * A * B'

        # the derivative of 1 / (2 lambda) || X - V ||_2^2 is 1/lambda * (X - V)
        t2 = 1 / lambda * (X - V)
        
        newX = X - step * (t1 + t2)
        newX = adjust(newX)
        
        if proxObj(lambda, X, V) < eps
            return X
        end
        
        if proxObj(lambda, X, V) < proxObj(lambda, Xmin, V)
            Xmin = copy(X)
        end
        
        X = copy(newX)
    end
    
    return Xmin
    
end

prox (generic function with 1 method)

##### As a sanity check, make sure that our gradient steps indeed decrease loss

In [14]:
Theta = adjust(rand(d1,d2))

println("Hidden loss ", loss(ThetaS))
println("Current loss ", loss(Theta))

for stp = 1:10
    alpha = 0.05
    der = deltaF(Theta)

    newTheta = Theta - alpha * der
    newTheta = adjust(newTheta)

    println("Loss after ", stp, " steps: ", loss(newTheta))
    
    Theta = copy(newTheta)
end

Hidden loss 0.39587472868210566
Current loss 0.7871777566811669
Loss after 1 steps: 0.7549552776556652
Loss after 2 steps: 0.7240941139300339
Loss after 3 steps: 0.6946391989696824
Loss after 4 steps: 0.666617723568723
Loss after 5 steps: 0.6400403102197109
Loss after 6 steps: 0.6149024316085975
Loss after 7 steps: 0.5911860132867398
Loss after 8 steps: 0.5688611687102807
Loss after 9 steps: 0.5478880188733919
Loss after 10 steps: 0.5282185458094266


#### Let's test out the algorithm

In [11]:
curX = adjust(rand(d1,d2))
lambda = 0.1

for steps = 1:100
    # first, go in the direction of F
    interX = curX - lambda * deltaF(curX)
    newX = prox(lambda, interX)
    
    if steps % 10 == 0
        println("f + g = ", loss(curX) + lambda * nucNorm(curX))
    #     println("f = ", loss(curX))
    #     println("g = ", lambda * nucNorm(curX))
    #     println()
    end
    
    curX = copy(newX)
end

f + g = 0.681113129753132
f + g = 0.4749330301788126
f + g = 0.4107402726800853
f + g = 0.38946601515297535
f + g = 0.382164842661707
f + g = 0.3795753705696425
f + g = 0.37875050542011324
f + g = 0.3782484797098058
f + g = 0.3780198231363411
f + g = 0.37764489218423236


In [12]:
println(loss(ThetaS) + lambda * nucNorm(ThetaS))

0.4958747286821057
